In [10]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

In [11]:
base_dir = os.getcwd()
data_folder = os.path.join(base_dir, "Dataset")

images_folder = os.path.join(data_folder, "Images")
crops_folder = os.path.join(data_folder, "Crops")
sample_testset = os.path.join(data_folder, "sample_testset")

sample_images = os.path.join(sample_testset, "images")
sample_crops = os.path.join(sample_testset, "crops")

In [12]:
def bruteForceSIFT(img1, img2):
    
    sift = cv2.xfeatures2d.SIFT_create()
    
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    good = []
    print(len(matches))
    for match1, match2 in matches:
        if match1.distance < 0.75 * match2.distance:
            good.append([match1])
            
    sift_matches = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, flags=2)
    
    sift_matches = cv2.cvtColor(sift_matches, cv2.COLOR_BGR2RGB)
    
    print(len(good))
    plt.imshow(sift_matches)
    plt.show()
    return (kp1, kp2)




In [13]:
def bruteForceORB(img1, img2):
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1,None)
    kp2, des2 = orb.detectAndCompute(img2,None)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    matches = bf.match(des1, des2)
    
    matches = sorted(matches, key = lambda x:x.distance)
    
    orb_matches = cv2.drawMatches(img1, kp1, img2, kp2,matches[:10], None,
                          flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
   
    orb_matches = cv2.cvtColor(orb_matches, cv2.COLOR_BGR2RGB)
    plt.imshow(orb_matches)
    plt.show()
    
    return matches[:10]

In [14]:
def FLANN(img1, img2):
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)    
    search_params = dict(checks=10)
    
    sift = cv2.xfeatures2d.SIFT_create()
    
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    
    matchesMask = [[0,0] for i in range(len(matches))]

    for i,(m,n) in enumerate(matches):
        if m.distance < 0.7*n.distance:
            matchesMask[i]=[1,0]

    draw_params = dict(matchColor = (0,255,0),
                   matchesMask = matchesMask,
                   flags = cv2.DrawMatchesFlags_DEFAULT)
    
    flann_matches = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
    
    orb_matches = cv2.cvtColor(flann_matches, cv2.COLOR_BGR2RGB)
    plt.imshow(orb_matches)
    plt.show()
    
    return matches[:10]

NameError: name 'crop_img' is not defined